In [1]:
import re as re
import subprocess as sub
import sys as sys
import numpy as np
import sympy as sympy
from Format_Inputs import g16_input_file

features {
  feature {
    key: "label"
    value {
      float_list {
        value: 0.244
      }
    }
  }
  feature {
    key: "features"
    value {
      float_list {
        value: 0
        value: 3.75
        value: 8
        value: 0.097
        value: 0.055
        value: 498
        value: 4.62
        value: 2.91
      }
    }
  }
}

features {
  feature {
    key: "label"
    value {
      float_list {
        value: 3.15
      }
    }
  }
  feature {
    key: "features"
    value {
      float_list {
        value: 2.5
        value: 2.99
        value: 57
        value: 0.068
        value: -0.19
        value: 1.18
        value: 10.98
        value: 2.8
      }
    }
  }
}

features {
  feature {
    key: "label"
    value {
      float_list {
        value: 0.88
      }
    }
  }
  feature {
    key: "features"
    value {
      float_list {
        value: 3.1
        value: 3.5
        value: 8
        value: 590
        value: 0.00024
        value: 0.0543
        

In [2]:
def not_real_val(output_val):
    if str(output_val) == str(1e400*0):
        NAN = True
    else:
        NAN = False
    return NAN

In [13]:
task_file = g16_input_file(seed_structure_directory = "/users/haydenprescott/documents", molecular_formula = "La3O4", job_type = "ground_state_energy_calculation", input_directory = "/users/haydenprescott/documents", output_directory = "/users/haydenprescott/documents/gaussian_test_data", AO_basis = " ", polarization = None, diffuse = None, job_title = "seed structure ground state", SCF_optimization = 'tight', csv_filepath = csv_filepath, record_filepath = record_filepath, record_filename = record_filename, element = None, n_CPU = 2, allowed_memory = "1000MB")
molecule_filename = sys.argv[1]
molecule_filename = task_file.generate_input_file(molecular_formula = task_file.molecular_formula, job_type = task_file.job_type, input_directory = task_file.input_directory, output_directory = task_file.output_directory, seed_structure_directory = task_file.seed_structure_directory, csv_filepath = task_file.csv_filepath, record_filepath = task_file.record_filepath, record_filename = task_file.record_filename, n_CPU = task_file.n_CPU, allowed_memory = task_file.allowed_memory)
output_filename = molecule_filename + "_output.txt"

def write_energy_data(output_filename):
    i = 0
    output = open(output_filename, "w+")
    while True:
        convert_to_checkpoint = "formchk %s_%i.chk %s_%i.fchk"%(molecule_filename, i, molecule_filename, i)
        try:
            open(("%s_%i.chk"%(molecule_filename,i)))
        except IOError:
            break
        if (sub.call(convert_to_checkpoint, shell = True)) == 0:
            output_file = open(output_filename + "_%i.fchk"%i, "r")
            n_valence_electrons = 0
            MOs = []
            parse_line = True
            j = 0
            placeholder_reg_exp = re.compile("Number of electrons\s+I\s+(\d+)")
            for readout in output_file:
                if "Alpha Orbital Energies" in readout:
                    parse_line = True
                    j = 0
                if parse_line == True:
                    j += 1
                if parse_line == True and j >= 2:
                    try:
                        MOs.extend([float(s) for s in readout.split()])
                    except:
                        parse_line = False
                match_found = placeholder_reg_exp.search(readout)
                if type(match_found) != None:
                    n_valence_electrons = int(match_found.group(1))
        
            log_output = open(output_filename+"_%i.log"%i)
            reg_exp_exc = re.compile("Excited State\W+1:.*[^-]([+-]?\d+\.\d+)\W+eV")
            reg_exp_ground = re.compile("Excited State\W+1:.*[^-]([+-]?\d+\.\d+)\W+eV")
        	#matches the energy of the first exitation state
            NAN_flag = float('nan');
            for readout in log_output:
                match_found_exc = reg_exp_exc.search(readout)
                match_found_ground = reg_exp_ground.search(readout)
                if type(match_found_exc) != None:
                    first_exc_state = float(match_found_exc.group(1))
                if isNaN(first_exc_state) == True:
                    pass
                if type(match_found_ground) != None:
                    ground_state_ener = float(match_found_ground.group(1))
                if isNaN(ground_state_ener) == True:
                    pass
        

        elif (sub.call(convert_to_checkpoint, shell = True)) != 0:
            break

        output.write("%i %.10e %.10e\n"%(i,orbs[electrons/2-1]-orbs[electrons/2],first_exc_state,ground_state_ener)) 
    	#the homo/lumo orbitals are assumed to be at halfe the number of electrons
	
        i += 1
    output.close()
	




In [ ]:
n = 100
s = 0
k_0 = 0.4
error_bound = 1e-2
target = 0.17
summation = 0
z = sympy.symbols('z', real = True)
pdf = (1/np.sqrt(2 * np.pi)) * sympy.exp(-z**2/2)
z_points = np.arange(0, k_0, k_0/n)
for i in np.arange(0, len(z_points)):
    current_sum = (k_0/(n + s)) * pdf.replace(z, (z_points[i] - z_points[i-1])/2)
    summation += current_sum

while abs(summation - target) > error_bound:
    s += 1
    k_0 += 1/100
    z_points = np.arange(0, k_0, k_0/n)
    current_sum = 0
    for i in np.arange(0, len(z_points)):
        current_sum = (k_0/(n + s)) * pdf.replace(z, (z_points[i] - z_points[i-1])/2)
        summation += current_sum
    
print(summation, k_0)

    